In [2]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import json
import numpy as np
import time
import random
import json
from concurrent.futures import ThreadPoolExecutor, as_completed

In [1]:
topics = [
    'https://gamersantai.com/category/berita/',
    'https://gamersantai.com/category/temuramah/',
    'https://gamersantai.com/category/review/'
]

In [3]:
r = requests.get(topics[0])
soup = BeautifulSoup(r.content, "lxml")

In [7]:
nums = []
a = soup.find_all('a', {'class': 'page_number'})
for a_ in a:
    try:
        nums.append(int(a_.text.replace(',','')))
    except:
        pass
    
if len(nums):
    page_num = max(nums)
else:
    page_num = 1
    
page_num

1290

In [9]:
h3 = soup.find_all('h3', {'class': 'jeg_post_title'})
a = [h3_.find('a').get('href') for h3_ in h3]
a

['https://gamersantai.com/barbie-review-aura-seronok-yang-menyerlah-riang-di-sebalik-krisis-kehidupan/',
 'https://gamersantai.com/ryu-ga-gotoku-studio-bincang-masa-depan-francais-yakuza-like-a-dragon-kisah-cinta-kazuma-kiryu-interview/',
 'https://gamersantai.com/one-piece-apakah-inspirasi-luffy-gear-5-sun-god-nika/',
 'https://gamersantai.com/review-rog-flow-x13-laptop-gaming-nipis-ringan-berprestasi-tinggi/',
 'https://gamersantai.com/jujutsu-kaisen-kuasa-gojo-yang-masih-belum-diterokai-dalam-musim-2/',
 'https://gamersantai.com/game-aksi-multiplayer-one-punch-man-world-akan-datang-ke-sea-mungkin-keluar-tahun-ini/',
 'https://gamersantai.com/guardians-of-the-galaxy-vol-3-bakal-tersedia-di-disney-pada-2-ogos-ini/',
 'https://gamersantai.com/persona-5-tactica-selami-sistem-battle-unik-mekanik-game/',
 'https://gamersantai.com/infinity-strash-dragon-quest-the-adventure-of-dai-kelebihan-setiap-watak/',
 'https://gamersantai.com/konsol-handheld-bakal-perlukan-bateri-boleh-diganti-pada-ta

In [10]:
!rm -rf url
!mkdir url

In [11]:
def crawl(url, filename):
    if os.path.exists(filename):
        return

    r = requests.get(url)
    soup = BeautifulSoup(r.content, "lxml")
    h3 = soup.find_all('h3', {'class': 'jeg_post_title'})
    a = [h3_.find('a').get('href') for h3_ in h3]

    with open(filename, 'w') as fopen:
        json.dump(a, fopen)

In [12]:
max_worker = 10

for t in topics:
    print(t)
    t_ = t.split('com/')[1].replace('/', '-')
    
    r = requests.get(t)
    soup = BeautifulSoup(r.content, "lxml")
    
    nums = []
    a = soup.find_all('a', {'class': 'page_number'})
    for a_ in a:
        try:
            nums.append(int(a_.text.replace(',','')))
        except:
            pass

    if len(nums):
        page_num = max(nums)
    else:
        page_num = 1
        
    for i in tqdm(range(1, page_num + 1, max_worker)):
        a = np.arange(i, i + max_worker)
        urls = [(f'{t}page/{k}', os.path.join('url', f'{t_}-{k}.json')) for k in a]
        
        with ThreadPoolExecutor(max_workers=max_worker) as executor:
            futures = {executor.submit(crawl, url[0], url[1]): url for url in urls}
            
            for future in as_completed(futures):
                future.result()

https://gamersantai.com/category/berita/


100%|███████████████████████████████████████| 129/129 [1:08:11<00:00, 31.72s/it]


https://gamersantai.com/category/temuramah/


100%|█████████████████████████████████████████████| 4/4 [00:53<00:00, 13.30s/it]


https://gamersantai.com/category/review/


100%|█████████████████████████████████████████████| 9/9 [02:38<00:00, 17.62s/it]
